In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

#!pip3 install itables
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=False)

from itables.sample_dfs import get_indicators
from itables.downsample import nbytes
import itables.options as opt

opt.lengthMenu = [20, 50, 100, 200, 500]
#opt.maxBytes = 0

df = get_indicators()

In [ ]:
import pandas as pd
# Set the display options
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.expand_frame_repr', False)  # Do not wrap DataFrame
pd.set_option('max_colwidth', None)  # Display full column width

In [ ]:
#!pip3 install attrs
#!pip3 install --upgrade numpy==1.24
#!pip3 install numba #==0.53
#!pip3 install --upgrade guesslang


In [ ]:
#!pip3 install --upgrade pandas
#!pip3 install --upgrade nltk
#!pip3 install --upgrade textblob
#!pip3 install ipython-autotime
#!pip3 install umap-learn[plot]
#!pip3 install hdbscan
#!pip3 uninstall hdbscan --yes
#!pip3 install hdbscan --no-cache-dir --no-binary :all:
#!python -m pip install hdbscan --no-cache-dir --no-binary :all:
import nltk
#nltk.download('punkt')
#nltk.download('wordnet')
#nltk.download('omw-1.4')
#nltk.download('brown')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('vader_lexicon')
#!pip3 install --upgrade requests
#!pip3 install sentence_transformers
#!pip3 install tensorflow
#!pip3 install tensorflow_hub
#!pip3 install --upgrade tensorflow #==2.9.0
#!pip3 install spaCy
#!python3 -m spacy download en_core_web_sm
#!pip3 install pandarallel
#!pip3 install imagehash
#!pip3 install simplejson

###plots
#!pip3 install matplotlib
#!pip3 install datashader
#!pip3 install bokeh
#!pip3 install holoviews
#!pip3 install pyenchant

In [ ]:
import pandas as pd
import numpy as np
import re
from nltk.stem import WordNetLemmatizer
import nltk
from textblob import TextBlob
from tqdm.notebook import tqdm
tqdm.pandas()
import requests
from pandarallel import pandarallel
import umap.umap_ as umap
import hdbscan
import imagehash
import spacy
from collections import Counter
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import random
import itertools
import json
#from google.colab import drive
#import simplejson


# to plot
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import LogNorm
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
%matplotlib inline
#import umap.plot
import random
import enchant
from numpy.linalg import norm

#topic modeling
import re
from nltk.corpus import brown
from gensim import models, corpora
from nltk import word_tokenize
from nltk.corpus import stopwords
import nltk
#nltk.download()

In [ ]:
# ML
import tensorflow as tf
import tensorflow_hub as hub
from sentence_transformers import SentenceTransformer
from scipy import spatial

# Functions

## ML Functions

In [ ]:
def embed(model, model_type, all_sentences):
  """
  wrapper function for generating message embeddings
  """
  if model_type == 'use':
        embeddings = model(list(all_sentences))
  elif model_type == 'sentence transformer':
        embeddings = model.encode(list(all_sentences),show_progress_bar=True)
    
  return embeddings

In [ ]:
def vectorToColumn(df,vectors,column_name):
    df[column_name]=0
    df[column_name]=df[column_name].astype(object)
    df.reset_index(drop=True,inplace=True)
    indices=df.index
    for i in indices:
        df.at[i, column_name]=vectors[i]
    return df

In [ ]:
nlp=spacy.load('en_core_web_sm')

def extract_labels(category_docs):
    """
    Extract labels from documents in the same cluster by concatenating
    most common verbs, ojects, and nouns
    """

    verbs = []
    dobjs = []
    nouns = []
    adjs = []
    
    verb = ''
    dobj = ''
    noun1 = ''
    noun2 = ''

    # for each document, append verbs, dobs, nouns, and adjectives to 
    # running lists for whole cluster
    for i in range(len(category_docs)):
        doc = nlp(category_docs[i])
        for token in doc:
            if token.is_stop==False:
                if token.dep_ == 'ROOT':
                    verbs.append(token.text.lower())

                elif token.dep_=='dobj':
                    dobjs.append(token.text.lower())

                elif token.pos_=='NOUN':
                    nouns.append(token.text.lower())
                    
                elif token.pos_=='ADJ':
                    adjs.append(token.text.lower())

    verbs = Counter(verbs)
    dobjs = Counter(dobjs)
    nouns = Counter(nouns)

    if len(verbs) > 0:
        verb = verbs.most_common(1)[0][0]

    if len(dobjs) > 0:
        dobj = dobjs.most_common(1)[0][0]

    if len(nouns) > 0:
        noun1 = nouns.most_common(1)[0][0]

    if len(set(nouns)) > 1:
        noun2 = nouns.most_common(2)[1][0]
    
    # concatenate the most common verb-dobj-noun1-noun2 (if they exist)
    label_words = [verb, dobj]
    
    for word in [noun1, noun2]:
        if word not in label_words:
            label_words.append(word)
    
    if '' in label_words:
        label_words.remove('')
    
    label = '_'.join(label_words)
    
    return label

In [ ]:
def generateUmapAndCluster(message_embeddings, n_neighbors, n_components, min_cluster_size, random_state = None, n_jobs=3):
    
    # reduces dimensionality
    reducer = (umap.UMAP(n_components = n_components, 
                                metric='cosine', 
                                random_state=random_state,
                                n_jobs = n_jobs,
                                verbose=True))

    umap_embeddings = reducer.fit_transform(message_embeddings)
    print(umap_embeddings.shape)
    
    clusters = hdbscan.HDBSCAN(min_cluster_size = min_cluster_size,
                        metric='euclidean', 
                        cluster_selection_method='eom', 
                        core_dist_n_jobs = multiprocessing.cpu_count()).fit_predict(umap_embeddings)
    return clusters, umap_embeddings



def generateUmap(message_embeddings, n_neighbors, n_components, random_state = None, n_jobs=3):
    # reduces dimensionality
    reducer = (umap.UMAP(n_components = n_components, 
                                metric='cosine', 
                                random_state=random_state,
                                n_jobs = n_jobs,
                                verbose=True))

    umap_embeddings = reducer.fit_transform(message_embeddings)
    print(umap_embeddings.shape)
    return umap_embeddings


import multiprocessing
def generateCluster(message_embeddings, n_neighbors, n_components, min_cluster_size, random_state = None):
    clusters = hdbscan.HDBSCAN(min_cluster_size = min_cluster_size,
                        metric='euclidean', 
                        cluster_selection_method='eom', 
                        core_dist_n_jobs = multiprocessing.cpu_count()).fit_predict(umap_embeddings)
    return clusters

### Clustering Functions

In [ ]:
def text_cluster_to_label(data_, model_type = ""):
    clusters_dict = dict()
    clusters_set = set(data_['cluster'+ model_type])
    for cluster_i in clusters_set:
        label_ = extract_labels(list(data_[data_['cluster'+ model_type]==cluster_i]['body']))
        print(label_)
        clusters_dict.update({cluster_i:label_})
    return data_['cluster'+ model_type].map(clusters_dict),clusters_dict

# +++++++++++++++
# START PROGRAM
# +++++++++++++++

In [ ]:
path_inputs = "/"
path_outputs = "/"
path_figures = "/"
all_forums = [1,2,3] # this list contains the numbers of the subforums

forums_to_process = all_forums

## Load THREADS into panda dataframe

Json file contains data in the format: 

    {
        "original_text": " ", 
        "translation": " ", 
        "language": " ", 
        "timestamp": "2000-01-31 12:00:00", 
        "paragraph_index": 0, 
        "id_thread": 1, 
        "id_post": 1
    }

In [ ]:
def forumdataToDataframe(forum_nr):
    dataframe_all_data = pd.read_json(f'{path_inputs}translations/{str(forum_nr)}_all_translations.json',  lines=True, encoding='utf-8')
    dataframe_all_data['timestamp'] = pd.to_datetime(dataframe_all_data['timestamp'])
    return dataframe_all_data

In [ ]:
df_dict= {}
dataframe_all_forums = pd.DataFrame()

# execute forumdataToDataframe, which gets the thread data from each json file and transforms is to a dataframe
# concatenate the new dataframe to one big dataframe
for forum_id in forums_to_process:
    try:
        df_dict[forum_id] = forumdataToDataframe(forum_id)
        df_dict[forum_id]["forum_id"] = forum_id
        df_userids= pd.read_csv(f'{path_inputs}userids/{forum_id}_df_userids_postids.csv')
        df_dict[forum_id] = pd.merge(df_dict[forum_id], df_userids, on = "id_post", how = "left")
    except Exception as e:
        print(f"Problems with {forum_id}. Exception: {e}")

for forum_id in forums_to_process:
    try:
        dataframe_all_forums = pd.concat([dataframe_all_forums, df_dict[forum_id]],ignore_index=True, axis=0)
        print(f"Forums {forum_id} successful")
    except Exception as e:
        print(f"Forum {forum_id} not merged. Exception: {e}")

In [ ]:
del df_dict # to free memory

In [ ]:
dataframe_all_forums = dataframe_all_forums.rename(columns={'translation': 'body'})

## Only keep first post

In [ ]:
min_sub_id = dataframe_all_forums.groupby("id_thread")["id_post"].min()
dataframe_all_forums = dataframe_all_forums[dataframe_all_forums["id_post"].isin(min_sub_id)]

In [ ]:
print(dataframe_all_forums['body'].isnull().sum()) # the amount of empty body lines
dataframe_all_forums = dataframe_all_forums.dropna(subset = ["body"])
dataframe_all_forums = dataframe_all_forums.reset_index(drop=True)

## Load HEADLINES to pandas

Headline json files contain: 

    {
        "original_text": " ", 
        "translation": " ", 
        "language": " ", 
        "timestamp": "2000-01-31 12:00:00", 
        "paragraph_index": -1, 
        "id_thread": 1419
    }
    
Sentence index is always -1, as it is a headline.

In [ ]:
def headlineDataToDataframe(forum_nr):
    dataframe_all_headlines = pd.read_json(f'{path_inputs}/translations/{str(forum_nr)}_headline_translations.json',  lines=True, encoding='utf-8')
    dataframe_all_headlines['timestamp'] = pd.to_datetime(dataframe_all_headlines['timestamp'])
    return dataframe_all_headlines

In [ ]:
dataframe_all_headlines = pd.DataFrame()
df_dict_headlines = {}

# execute forumdataToDataframe, which gets the threaddata from each csv file and transforms is to a dataframe
# concatenate the new dataframe to one big dataframe
for forum_id in forums_to_process:
    try:
        df_dict_headlines[forum_id] = headlineDataToDataframe(forum_id)
        df_dict_headlines[forum_id]["forum_id"] = forum_id
        df_userids= pd.read_csv(f'{path_inputs}userids/{forum_id}_df_userids_threadids.csv')
        df_dict_headlines[forum_id] = pd.merge(df_dict_headlines[forum_id], df_userids, on = "id_thread", how = "left")
    except Exception as e:
        print(f"Problems with {forum_id}. Exception: {e}")

for forum_id in forums_to_process:
    try:
        dataframe_all_headlines = pd.concat([dataframe_all_headlines, df_dict_headlines[forum_id]],ignore_index=True, axis=0)
        print(f"Headlines {forum_id} successful")
    except Exception as e:
        print(f"Headlines {forum_id} not merged. Exception: {e}")

In [ ]:
del df_dict_headlines # to free memory

In [ ]:
dataframe_all_headlines = dataframe_all_headlines.rename(columns={'translation': 'body'})
dataframe_all_headlines["id_post"] = -1

### Remove non en or non ru text

In [ ]:
dataframe_all_forums = dataframe_all_forums[dataframe_all_forums["language"].isin(["en", "ru"])]
dataframe_all_forums = dataframe_all_forums.reset_index(drop=True)

In [ ]:
dataframe_all_headlines = dataframe_all_headlines[dataframe_all_headlines["language"].isin(["en", "ru"])]
dataframe_all_headlines = dataframe_all_headlines.reset_index(drop=True)

### Text preparation & cleaning

In [ ]:
punctuations = """!"%()'*+,*/:#»«";“<=>?[\]^`”{.}~-_|$@"""
numbers = "0123456789"
def clean_text_local(row):
    
    row=re.sub("-?NEWLINE_TOKEN", " ",row)
    row=re.sub("TAB_TOKEN", " ",row)
    row=re.sub("Alternate option=", "",row)
    row=re.sub('RT','',row)
    row = row.lower()

    row=re.sub('\x0c',' ',row)
    row=re.sub('\u200c',' ',row)
    row=re.sub('\\\\n',' ',row)
    row=re.sub('\n',' ',row)

    row=re.sub("@[A-Za-z0-9]+","",row)
    row=re.sub("http\S+|www.\S+","",row) # removes the urls
    row = re.sub(r'<.*?>', '', row)
    row=re.sub("&amp","&",row)
    no_punct = ""
    for char in row:
        if char not in punctuations:
            no_punct = no_punct + char
    return no_punct


def check_if_char_is_letter(row): #To check if there are only numbers or empty string in one, returns None if string only Numbers or empty
    for char in row:
        if char.isalpha() == True: 
            return True
    return None

def check_percent_nonalphabet(row, threshold = 100):
    amount_of_non_alphabet = 0
    for char in row:
        if char.isalpha() == False:
            amount_of_non_alphabet = amount_of_non_alphabet + 1
    if (amount_of_non_alphabet/len(row))*100 >= threshold:  #check if min 90% are non-alpha char
        return None
    else:
        return row

def check_punctuation_percent_nr(row_text, threshold):
    amount_of_punctuations = 0
    row_text = str(row_text)
    for char in row_text:
        if char in punctuations:
            amount_of_punctuations = amount_of_punctuations + 1
    if (amount_of_punctuations) == 0 or len(row_text) == 0: return True
    if (amount_of_punctuations/len(row_text))*100 > threshold:  #check if max threshold% are punctuations
        return False
    else:
        return True

def cleanRowsWithPercentageOfPunctuations(dataframe, threshold):
    dropped_rows = []
    for index, row in dataframe.iterrows():
        if check_punctuation_percent_nr(row['body'], threshold) == True: # more than threshold are punctuations => sorted out
            #print(row["body"])
            dropped_rows.append(row["body"])
            dataframe.drop([index], inplace = True)
    return dataframe, dropped_rows

def check_if_sentence_long_enough(text):
    counted_spaces = 0
    for i in text:
        if i == " ":
            counted_spaces = counted_spaces + 1
    if (len(text) > 29) or (counted_spaces > 4):
        return text
    else:
        print(f"Sorted out: {text}")
        return ""

### Cleaning Forum TEXT and HEADLINE

In [ ]:
def apply_cleaning_to_row(row_text, punctuation_threshold = 20):
    """Returns the cleaned row or None"""
    # 1. check amount of punctuations
    if check_punctuation_percent_nr(row_text, punctuation_threshold) == True:
        # 2. Check if sentences is long enough and return "" if its not
        row_text = check_if_sentence_long_enough(row_text)
        # 3. clean punctuations & lower
        row_text = clean_text_local(row_text)
        # 4. delete rows that do not contain any alphabetic letter (eg. only numbers, or empty)
        if check_if_char_is_letter(row_text) == True: #continue if alphabetic letters included, else return none
            return row_text
        else:
            return None
    else: 
        return None
    

def apply_cleaning_to_HEADLINE(row_text):
    """Returns the cleaned row or None"""
    # 3. clean punctuations & lower
    row_text = clean_text_local(row_text)
    # 4. delete rows that do not contain any alphabetic letter (eg. only numbers, or empty)
    if check_if_char_is_letter(row_text) == True: #continue if alphabetic letters included, else return none
        return row_text
    else:
        return None

In [ ]:
punctuation_threshold = 12
dataframe_all_forums['body'] = dataframe_all_forums['body'].progress_apply(lambda text: apply_cleaning_to_row(text, punctuation_threshold))
# Drop rows where the text is None
dataframe_all_forums.dropna(subset=['body'], inplace=True)

In [ ]:
dataframe_all_headlines['body'] = dataframe_all_headlines['body'].progress_apply(lambda text: apply_cleaning_to_HEADLINE(text))
# Drop rows where the text is None
dataframe_all_headlines.dropna(subset=['body'], inplace=True)

## Put both together

In [ ]:
dataframe_all_forums = pd.concat([dataframe_all_forums, dataframe_all_headlines],ignore_index=True, axis=0)
dataframe_all_forums= dataframe_all_forums.sort_values(['id_thread', 'id_post', 'sentence_index'], ascending=[True, True, True])
dataframe_all_forums = dataframe_all_forums.reset_index(drop=True)

## Splitting into language

In [ ]:
dataframe_all_forums_ru = dataframe_all_forums[dataframe_all_forums["language"] == "ru"]
dataframe_all_forums_en = dataframe_all_forums[dataframe_all_forums["language"] == "en"]

## Generate embeddings

In [ ]:
model_st1 = SentenceTransformer('all-mpnet-base-v2', device = "cuda") #sentence-BERT model

embeddings_st1_ru = embed(model=model_st1, model_type='sentence transformer', all_sentences=dataframe_all_forums_ru['body']) #round(len(list(dataframe_all_forums["body"]))/8000)) # devide the set so that one partition has about 5000-10000 partitions
dataframe_all_forums_ru=vectorToColumn(df=dataframe_all_forums_ru, vectors=embeddings_st1_ru, column_name='all_mpnet_base_v2')

embeddings_st1_en = embed(model=model_st1, model_type='sentence transformer', all_sentences=dataframe_all_forums_en['body']) #round(len(list(dataframe_all_forums["body"]))/8000)) # devide the set so that one partition has about 5000-10000 partitions
dataframe_all_forums_en = vectorToColumn(df=dataframe_all_forums_en, vectors=embeddings_st1_en, column_name='all_mpnet_base_v2')

## Reduce dimension and cluster

In [ ]:
umap_embeddings_ru = generateUmap(list(dataframe_all_forums_ru["all_mpnet_base_v2"]), n_neighbors = 6,  n_components= 9, random_state = 1, n_jobs=3)
dataframe_all_forums_ru = vectorToColumn(df = dataframe_all_forums_ru, vectors = umap_embeddings_ru, column_name = 'all-mpnet-base-v2_reduced')

umap_embeddings_en = generateUmap(list(dataframe_all_forums_en["all_mpnet_base_v2"]), n_neighbors = 6,  n_components= 9, random_state = 1, n_jobs=3)
dataframe_all_forums_en = vectorToColumn(df = dataframe_all_forums_en, vectors = umap_embeddings_en, column_name = 'all-mpnet-base-v2_reduced')

In [ ]:
dataframe_all_forums_ru['cluster'] = generateCluster(embeddings_st1_ru, n_neighbors = 6,  n_components= 9, random_state = 1, min_cluster_size = 24)
dataframe_all_forums_en['cluster'] = generateCluster(embeddings_st1_en, n_neighbors = 6,  n_components= 9, random_state = 1, min_cluster_size = 24)

## Generating cluster_labels

In [ ]:
dataframe_all_forums_ru['cluster_label'], clusters_dict_all_forums_ru = text_cluster_to_label(data_=dataframe_all_forums_ru)
dataframe_all_forums_en['cluster_label'], clusters_dict_all_forums_en = text_cluster_to_label(data_=dataframe_all_forums_en)

# SAVE DATAFRAMES

In [ ]:
dataframe_all_forums_ru.to_csv(f'{path_outputs}dataframe_all_forums_clustering_and_labeling_completed_ru_only.csv')
dataframe_all_forums_en.to_csv(f'{path_outputs}dataframe_all_forums_clustering_and_labeling_completed_en_only.csv')